In [1]:
import torch
import torchvision.models as models
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


In [2]:
# train_dataset = CustomDataset()
# val_dataset = CustomDataset()

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [3]:
# Define the transformations to apply to the CIFAR-10 data
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

download = True
# Define the training and test datasets
train_dataset = datasets.Flowers102(root='./data', split="train", download=download, transform=data_transforms)
test_dataset = datasets.Flowers102(root='./data', split="test", download=download, transform=data_transforms)

# Define the dataloaders to load the data in batches during training and testing
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [4]:
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

Number of training batches: 16
Number of testing batches: 97


In [5]:
# the network to test
model = models.resnet18(pretrained=False)
num_classes = 102

learning_rate = 1e-3

# Modify the last fully connected layer
fc_input = model.fc.in_features
model.fc = nn.Linear(fc_input, num_classes)

# print(model)

# Step 5: Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/asagrawal/private/.env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asagrawal/private/.env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
num_params = sum(torch.numel(p) for p in model.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 11228838


In [7]:
log_dir = "./logs/flowers"  # Set the directory for storing the logs
writer = SummaryWriter(log_dir)

In [8]:
def model_eval(model, dataloader):
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [9]:
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

log_train_every = 1
log_test_every = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

#         if (i+1) % 100 == 0:
#             print(f"[Epoch: {epoch + 1}, Batch: {i + 1}] Loss: {running_loss/100:.3f}")
#             running_loss = 0.0
        
    # compute training & testing accuracy every couple of iterations        
    if (epoch+1) % log_train_every == 0:
        train_accuracy = model_eval(model, train_loader)

        # Log the loss
        #writer.add_scalar('Loss/train', loss.cpu().item(), epoch * len(train_loader) + i)

        # Log the training accuracy
        #writer.add_scalar('Accuracy/train', train_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Loss: {running_loss} Accuracy: {train_accuracy}")
        
    if (epoch+1) % log_test_every == 0:
        test_accuracy = model_eval(model, test_loader)

        # Log the test accuracy
        #writer.add_scalar('Accuracy/test', test_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Test Accuracy: {test_accuracy}")
        
writer.close()
print("Training finished.")

Epoch 1 : Loss: 73.1183590888977 Accuracy: 3.3333333333333335
Epoch 2 : Loss: 82.85324430465698 Accuracy: 2.9411764705882355
Epoch 3 : Loss: 71.90087747573853 Accuracy: 3.627450980392157
Epoch 4 : Loss: 68.51050806045532 Accuracy: 6.078431372549019
Epoch 5 : Loss: 64.83104801177979 Accuracy: 7.647058823529412
Epoch 5 : Test Accuracy: 5.838347698812815
Epoch 6 : Loss: 63.22922658920288 Accuracy: 8.137254901960784
Epoch 7 : Loss: 61.54592680931091 Accuracy: 9.411764705882353
Epoch 8 : Loss: 59.069161891937256 Accuracy: 12.647058823529411
Epoch 9 : Loss: 58.29528570175171 Accuracy: 13.03921568627451
Epoch 10 : Loss: 56.53870916366577 Accuracy: 12.352941176470589
Epoch 10 : Test Accuracy: 9.237274353553424
Epoch 11 : Loss: 56.48313641548157 Accuracy: 16.176470588235293
Epoch 12 : Loss: 53.14680480957031 Accuracy: 16.07843137254902
Epoch 13 : Loss: 52.86601114273071 Accuracy: 21.372549019607842
Epoch 14 : Loss: 50.15495276451111 Accuracy: 27.45098039215686
Epoch 15 : Loss: 46.42010879516601

## Curriculum Learning experiment 1

So.. The curriculum for the following training procedure is that we increase the number of classes progressively in the dataset. All original labels are shited by 1. So, label 0 becomes 1, 1 becomes 2.. and so on. 
We first start with 2 classes 0 and 1 where 0 represents rest of classes and 1 represents the class that was originally 0. Then we work with 3 classes after 2 epochs: 0(rest of the classes), 1 and 2.

In [10]:
class SubsetDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes, num_classes, train=True, total_classes=102):
        self.dataset = dataset
        self.classes = classes
        self.num_classes = num_classes
        self.train = train
        self.total_classes = total_classes

    def __getitem__(self, index):
        image, label = self.dataset[index]
        if not self.train:
            return image, (label+1)%self.total_classes# we would reserve 0 for non-existing classes
        
        if label in self.classes[:self.num_classes]:
            return image, (label+1)%self.total_classes
        else:
            return torch.zeros_like(image), 0# IMPORTANT CHOICE MADE HERE 

    def __len__(self):
        return len(self.dataset)

In [11]:
####### Reset the model
# the network to test
model = models.resnet18(pretrained=False)
num_classes = 102

learning_rate = 1e-3

# Modify the last fully connected layer
fc_input = model.fc.in_features
model.fc = nn.Linear(fc_input, num_classes)

# print(model)

# Step 5: Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

curriculum_epochs = 5
increment_step = 2
nclass = increment_step


log_train_every = 1
log_test_every = 5

test_dataset = SubsetDataset(train_dataset, list(range(num_classes)), num_classes, train=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=4, shuffle=True)

for epoch in range(num_epochs):
    running_loss = 0.0
    
    subset_classes = list(range(nclass))

    # Create the modified dataset
    subset_dataset = SubsetDataset(train_dataset, subset_classes, nclass)

    # Create the train loader using the modified dataset
    train_loader = torch.utils.data.DataLoader(subset_dataset, batch_size=batch_size, num_workers=4, shuffle=True)

    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

#         if (i+1) % 100 == 0:
#             print(f"[Epoch: {epoch + 1}, Batch: {i + 1}] Loss: {running_loss/100:.3f}")
#             running_loss = 0.0
        
    # compute training & testing accuracy every couple of iterations        
    if (epoch+1) % log_train_every == 0:
        train_accuracy = model_eval(model, train_loader)

        # Log the loss
        #writer.add_scalar('Loss/train', loss.cpu().item(), epoch * len(train_loader) + i)

        # Log the training accuracy
        #writer.add_scalar('Accuracy/train', train_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Loss: {running_loss} Accuracy: {train_accuracy}")
        
    if (epoch+1) % log_test_every == 0:
        test_accuracy = model_eval(model, test_loader)

        # Log the test accuracy
        #writer.add_scalar('Accuracy/test', test_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Test Accuracy: {test_accuracy}")
    
    nclass += increment_step

writer.close()
print("Training finished.")

Epoch 1 : Loss: 28.60373695503222 Accuracy: 99.01960784313725
Epoch 2 : Loss: 5.551096052600769 Accuracy: 96.76470588235294
Epoch 3 : Loss: 2.8948125168681145 Accuracy: 94.90196078431373
Epoch 4 : Loss: 3.673274911940098 Accuracy: 93.82352941176471
Epoch 5 : Loss: 4.751058906316757 Accuracy: 91.17647058823529
Epoch 5 : Test Accuracy: 0.9803921568627451
Epoch 6 : Loss: 5.875548914074898 Accuracy: 90.68627450980392
Epoch 7 : Loss: 6.875649452209473 Accuracy: 87.54901960784314
Epoch 8 : Loss: 7.982627183198929 Accuracy: 85.68627450980392
Epoch 9 : Loss: 8.9874746799469 Accuracy: 84.90196078431373
Epoch 10 : Loss: 10.013173878192902 Accuracy: 82.84313725490196
Epoch 10 : Test Accuracy: 2.450980392156863
Epoch 11 : Loss: 10.713112592697144 Accuracy: 80.58823529411765
Epoch 12 : Loss: 11.600902885198593 Accuracy: 79.80392156862744
Epoch 13 : Loss: 12.792977720499039 Accuracy: 77.15686274509804
Epoch 14 : Loss: 13.968411982059479 Accuracy: 76.86274509803921
Epoch 15 : Loss: 15.258355677127838